In [1]:
import pandas as pd
import duckdb as db

This notebook will be used to explore the data and to use this as a basis for creating DBT models to be used for data cleaning.

This uses duckdb sql for data wrangling and will be changed to postgres syntax later on

In [4]:
df = pd.read_parquet(r"C:\Users\USER\Desktop\local_bucket\RAW_2017\VRA_2017.parquet")

In [5]:
dim_airports = pd.read_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\vra_azu_airports_final.csv")
dim_aircrafts = pd.read_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\vra_azu_aircrafts.csv")
dim_line_types = pd.read_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\line_type_codes.csv")
dim_flight_situation = pd.read_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\flight_situation_codes.csv")
dim_flight_status = pd.read_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\flight_status.csv")


#### Filter to AZU flights only and regular flights only

In [6]:
stg_01_filter_records = db.sql(
    """
    with q1 as (
        select *
        from df
        where trim(airline_icao) = 'AZU' and trim(di_code) = '0'
        order by trim(flight_number) asc, trim(scheduled_departure) asc    
    ),
    q2 as (
    select
        trim(airline_icao) as airline_icao,
        trim(flight_number) as flight_number,
        trim(line_type_code) as line_type_code,
        trim(aircraft_icao) as aircraft_icao,
        trim(passenger_count) as passenger_count,
        trim(orig_airport_icao) as orig_airport_icao,
        trim(scheduled_departure) as scheduled_departure,
        trim(actual_departure) as actual_departure,
        trim(dest_airport_icao) as dest_airport_icao,
        trim(scheduled_arrival) as scheduled_arrival,
        trim(actual_arrival) as actual_arrival,
        trim(flight_status) as flight_status,
        trim(justification) as justification
    from q1       
    )
    select 
    distinct *
    from q2

    """
).df()

In [7]:
stg_01_filter_records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284521 entries, 0 to 284520
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   airline_icao         284521 non-null  object
 1   flight_number        284521 non-null  object
 2   line_type_code       284521 non-null  object
 3   aircraft_icao        284521 non-null  object
 4   passenger_count      284521 non-null  object
 5   orig_airport_icao    284521 non-null  object
 6   scheduled_departure  284521 non-null  object
 7   actual_departure     164503 non-null  object
 8   dest_airport_icao    284521 non-null  object
 9   scheduled_arrival    284521 non-null  object
 10  actual_arrival       164503 non-null  object
 11  flight_status        284521 non-null  object
 12  justification        214536 non-null  object
dtypes: object(13)
memory usage: 28.2+ MB


##### Modify actual_departure and actual_arrival columns if it is null and flight_status = 'REALIZADO'


1. If flight_status = `'REALIZADO'` and both `actual_departure` and `actual_arrival` is null, the value to be used will be the `scheduled_departure` and `scheduled_arrival`.

In [8]:
stg_02_modify_datetime = db.sql(
    """
    select
        *,
        case
            when actual_departure is Null and flight_status = 'REALIZADO' then scheduled_departure
            else actual_departure
        end as mod_actual_departure,
        case
            when actual_arrival is Null and flight_status = 'REALIZADO' then scheduled_arrival
            else actual_arrival
        end as mod_actual_arrival,
        case
            when actual_departure is Null and actual_arrival is Null and flight_status != 'CANCELADO' then 1
        else 0
        end as datetime_mod_flag,
    from stg_01_filter_records
    order by scheduled_departure asc
    """
).df()

In [9]:
stg_02_modify_datetime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284521 entries, 0 to 284520
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   airline_icao          284521 non-null  object
 1   flight_number         284521 non-null  object
 2   line_type_code        284521 non-null  object
 3   aircraft_icao         284521 non-null  object
 4   passenger_count       284521 non-null  object
 5   orig_airport_icao     284521 non-null  object
 6   scheduled_departure   284521 non-null  object
 7   actual_departure      164503 non-null  object
 8   dest_airport_icao     284521 non-null  object
 9   scheduled_arrival     284521 non-null  object
 10  actual_arrival        164503 non-null  object
 11  flight_status         284521 non-null  object
 12  justification         214536 non-null  object
 13  mod_actual_departure  234333 non-null  object
 14  mod_actual_arrival    234333 non-null  object
 15  datetime_mod_flag

In [10]:
stg_02_modify_datetime.head()

,airline_icao,flight_number,line_type_code,aircraft_icao,passenger_count,orig_airport_icao,scheduled_departure,actual_departure,dest_airport_icao,scheduled_arrival,actual_arrival,flight_status,justification,mod_actual_departure,mod_actual_arrival,datetime_mod_flag
0,AZU,4979,R,E190,118,SBMO,01/01/2017 00:35,None,SBRP,01/01/2017 03:25,None,CANCELADO,AUTORIZADO,None,None,0
1,AZU,4171,N,E195,118,SBBV,01/01/2017 02:20,01/01/2017 02:07,SBEG,01/01/2017 03:35,01/01/2017 03:26,REALIZADO,ANTECIPAÇÃO DE HORÁRIO AUTORIZADA,01/01/2017 02:07,01/01/2017 03:26,0
2,AZU,4299,R,A320,174,SBFZ,01/01/2017 02:20,None,SBKP,01/01/2017 05:45,None,REALIZADO,None,01/01/2017 02:20,01/01/2017 05:45,1
3,AZU,4479,E,E190,118,SBAR,01/01/2017 02:45,None,SBCF,01/01/2017 04:45,None,CANCELADO,AUTORIZADO,None,None,0
4,AZU,4010,N,A332,242,SBEG,01/01/2017 03:05,None,SBKP,01/01/2017 06:45,None,REALIZADO,None,01/01/2017 03:05,01/01/2017 06:45,1


In [11]:
"""
Notes:
1. strptime only works in duckdb, to process in postgres it will changed to:
TO_TIMESTAMP(column, 'DD/MM/YYYY HH24:MI')
2. Change 'CANCELADO' and 'REALIZADO' to english equivalent
"""

stg_03_convert_types_keys = db.sql(
    """
    with q1 as (
    select
        airline_icao::varchar                               as airline_icao,
        flight_number::varchar                              as flight_number,
        line_type_code::char                                as line_type_code,
        aircraft_icao::varchar                              as aircraft_icao,
        passenger_count::int                                as passenger_count,
        orig_airport_icao::varchar                          as orig_airport_icao,
        strptime(scheduled_departure,'%d/%m/%Y %H:%M')      as scheduled_departure,
        strptime(mod_actual_departure,'%d/%m/%Y %H:%M')     as actual_departure,
        dest_airport_icao::varchar                          as dest_airport_icao,
        strptime(scheduled_arrival,'%d/%m/%Y %H:%M')        as scheduled_arrival,
        strptime(mod_actual_arrival,'%d/%m/%Y %H:%M')       as actual_arrival,
        case
            when flight_status = 'CANCELADO' then 'Cancelled'
            else 'Accomplished'
        end                                                 as flight_status,
        justification::varchar                              as justification,
        datetime_mod_flag::int                              as mod_flag
    from stg_02_modify_datetime
    order by scheduled_departure asc
    ),
    q2 as (
    select
        distinct *
    from q1
    order by scheduled_departure asc
    ),
    
    q3 as (
        select
        md5(airline_icao||flight_number||scheduled_departure||scheduled_arrival||row_number() over()) as sk_id,
        *
    from q2
    )
    
    select *
    from q3

    """
).df()

In [12]:
stg_03_convert_types_keys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284521 entries, 0 to 284520
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   sk_id                284521 non-null  object        
 1   airline_icao         284521 non-null  object        
 2   flight_number        284521 non-null  object        
 3   line_type_code       284521 non-null  object        
 4   aircraft_icao        284521 non-null  object        
 5   passenger_count      284521 non-null  int32         
 6   orig_airport_icao    284521 non-null  object        
 7   scheduled_departure  284521 non-null  datetime64[us]
 8   actual_departure     234333 non-null  datetime64[us]
 9   dest_airport_icao    284521 non-null  object        
 10  scheduled_arrival    284521 non-null  datetime64[us]
 11  actual_arrival       234333 non-null  datetime64[us]
 12  flight_status        284521 non-null  object        
 13  justification 

In [13]:
stg_03_convert_types_keys.head()

,sk_id,airline_icao,flight_number,line_type_code,aircraft_icao,passenger_count,orig_airport_icao,scheduled_departure,actual_departure,dest_airport_icao,scheduled_arrival,actual_arrival,flight_status,justification,mod_flag
0,de3dfd1651eeb06ce472a9c04fa86612,AZU,4979,R,E190,118,SBMO,2017-01-01 00:35:00,NaT,SBRP,2017-01-01 03:25:00,NaT,Cancelled,AUTORIZADO,0
1,8a66c4cc68cac1c20d71ac62c25b7d58,AZU,4299,R,A320,174,SBFZ,2017-01-01 02:20:00,2017-01-01 02:20:00,SBKP,2017-01-01 05:45:00,2017-01-01 05:45:00,Accomplished,None,1
2,42b9b793b5af8b0fea1f21ea1020be73,AZU,4171,N,E195,118,SBBV,2017-01-01 02:20:00,2017-01-01 02:07:00,SBEG,2017-01-01 03:35:00,2017-01-01 03:26:00,Accomplished,ANTECIPAÇÃO DE HORÁRIO AUTORIZADA,0
3,c6f430d81d0c35fcefa4505378a55b48,AZU,4479,E,E190,118,SBAR,2017-01-01 02:45:00,NaT,SBCF,2017-01-01 04:45:00,NaT,Cancelled,AUTORIZADO,0
4,80d784cf5a6ac50277e0b56bc9dcc239,AZU,4010,N,A332,242,SBEG,2017-01-01 03:05:00,2017-01-01 03:05:00,SBKP,2017-01-01 06:45:00,2017-01-01 06:45:00,Accomplished,None,1


In [14]:
stg_04_dim_justification_recs = db.sql(
    """
    select
        sk_id,
        coalesce(justification,'No Justification') as justification
    from stg_03_convert_types_keys
    """
).df()

In [15]:
stg_04_dim_justification_recs.head()

,sk_id,justification
0,de3dfd1651eeb06ce472a9c04fa86612,AUTORIZADO
1,8a66c4cc68cac1c20d71ac62c25b7d58,No Justification
2,42b9b793b5af8b0fea1f21ea1020be73,ANTECIPAÇÃO DE HORÁRIO AUTORIZADA
3,c6f430d81d0c35fcefa4505378a55b48,AUTORIZADO
4,80d784cf5a6ac50277e0b56bc9dcc239,No Justification


In [16]:
stg_05_dim_datetime = db.sql(
    """
    with q1 as (
    select
    sk_id,
    scheduled_departure,
    actual_departure,
    coalesce(extract(epoch from (actual_departure - scheduled_departure)) / 60, 0)::numeric(10,2) 	as departure_time_diff,
    case
        when coalesce(extract(epoch from (actual_departure - scheduled_departure)) / 60, 0) < 0.0 then 'Advanced'
        when coalesce(extract(epoch from (actual_departure - scheduled_departure)) / 60, 0) <= 30.0 then 'Punctual'
        when coalesce(extract(epoch from (actual_departure - scheduled_departure)) / 60, 0) <= 60.0 then 'Delay 30-60'
        when coalesce(extract(epoch from (actual_departure - scheduled_departure)) / 60, 0) <= 120.0 then 'Delay 60-120'
        when coalesce(extract(epoch from (actual_departure - scheduled_departure)) / 60, 0) <= 240.0 then 'Delay 120-240'
        else 'Delay > 240'
    end as departure_situation,
	-- arrival timestamp
    scheduled_arrival,
    actual_arrival,
    coalesce(extract(epoch from (actual_arrival - scheduled_arrival)) / 60, 0)::numeric(10,2) 		as arrival_time_diff,
	case
        when coalesce(extract(epoch from (actual_arrival - scheduled_arrival)) / 60, 0) < 0.0 then 'Advanced'
        when coalesce(extract(epoch from (actual_arrival - scheduled_arrival)) / 60, 0) <= 30.0 then 'Punctual'
        when coalesce(extract(epoch from (actual_arrival - scheduled_arrival)) / 60, 0) <= 60.0 then 'Delay 30-60'
        when coalesce(extract(epoch from (actual_arrival - scheduled_arrival)) / 60, 0) <= 120.0 then 'Delay 60-120'
        when coalesce(extract(epoch from (actual_arrival - scheduled_arrival)) / 60, 0) <= 240.0 then 'Delay 120-240'
        else 'Delay > 240'
    end as arrival_situation,
    coalesce(extract(epoch from (scheduled_arrival - scheduled_departure)) / 60, 0)::numeric(10,2) 	as expected_flight_duration,
    coalesce(extract(epoch from (actual_arrival - actual_departure)) / 60, 0)::numeric(10,2) 			as actual_flight_duration    
    from stg_03_convert_types_keys
    ),
    q2 as (
        select
            a.sk_id,
            a.scheduled_departure,
            a.actual_departure,
            a.departure_time_diff,           
            b1.situation_id as departure_situation,
            a.scheduled_arrival,
            a.actual_arrival,
            a.arrival_time_diff,
            b2.situation_id as arrival_situation,
            a.expected_flight_duration,
            a.actual_flight_duration
        from q1 as a
        left join dim_flight_situation as b1 on a.departure_situation = b1.situation_short_desc
        left join dim_flight_situation as b2 on a.arrival_situation = b2.situation_short_desc
    )
    
    select *
    from q2
    order by scheduled_departure asc
    """
).df()

In [17]:
stg_05_dim_datetime.head()

,sk_id,scheduled_departure,actual_departure,departure_time_diff,departure_situation,scheduled_arrival,actual_arrival,arrival_time_diff,arrival_situation,expected_flight_duration,actual_flight_duration
0,de3dfd1651eeb06ce472a9c04fa86612,2017-01-01 00:35:00,NaT,0.0,P,2017-01-01 03:25:00,NaT,0.0,P,170.0,0.0
1,8a66c4cc68cac1c20d71ac62c25b7d58,2017-01-01 02:20:00,2017-01-01 02:20:00,0.0,P,2017-01-01 05:45:00,2017-01-01 05:45:00,0.0,P,205.0,205.0
2,42b9b793b5af8b0fea1f21ea1020be73,2017-01-01 02:20:00,2017-01-01 02:07:00,-13.0,A,2017-01-01 03:35:00,2017-01-01 03:26:00,-9.0,A,75.0,79.0
3,c6f430d81d0c35fcefa4505378a55b48,2017-01-01 02:45:00,NaT,0.0,P,2017-01-01 04:45:00,NaT,0.0,P,120.0,0.0
4,80d784cf5a6ac50277e0b56bc9dcc239,2017-01-01 03:05:00,2017-01-01 03:05:00,0.0,P,2017-01-01 06:45:00,2017-01-01 06:45:00,0.0,P,220.0,220.0


In [18]:
stg_05_dim_datetime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284521 entries, 0 to 284520
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   sk_id                     284521 non-null  object        
 1   scheduled_departure       284521 non-null  datetime64[us]
 2   actual_departure          234333 non-null  datetime64[us]
 3   departure_time_diff       284521 non-null  float64       
 4   departure_situation       284521 non-null  object        
 5   scheduled_arrival         284521 non-null  datetime64[us]
 6   actual_arrival            234333 non-null  datetime64[us]
 7   arrival_time_diff         284521 non-null  float64       
 8   arrival_situation         284521 non-null  object        
 9   expected_flight_duration  284521 non-null  float64       
 10  actual_flight_duration    284521 non-null  float64       
dtypes: datetime64[us](4), float64(4), object(3)
memory usage: 23.9+ M

In [19]:
stg_06_dim_datetime_breakdown = db.sql(
    """
	select
    	sk_id,
    	cast(extract(day from scheduled_departure) as int) as scheduled_departure_day,
    	cast(extract(month from scheduled_departure) as int) as scheduled_departure_month,
    	cast(extract(year from scheduled_departure) as int) as scheduled_departure_year,
    	cast(extract(dow from scheduled_departure) as int) as scheduled_departure_dow,
    	strftime('%B', scheduled_departure) as scheduled_departure_month_name,
    	strftime('%A', scheduled_departure) as scheduled_departure_day_name,
    	cast(scheduled_departure as date) as scheduled_departure_date,

    	-- actual departure timestamp breakdown
    	cast(extract(day from actual_departure) as int) as actual_departure_day,
    	cast(extract(month from actual_departure) as int) as actual_departure_month,
    	cast(extract(year from actual_departure) as int) as actual_departure_year,
    	cast(extract(dow from actual_departure) as int) as actual_departure_dow,
    	strftime('%B', actual_departure) as actual_departure_month_name,
    	strftime('%A', actual_departure) as actual_departure_day_name,
    	cast(actual_departure as date) as actual_departure_date,

    	-- scheduled arrival timestamp breakdown
    	cast(extract(day from scheduled_arrival) as int) as scheduled_arrival_day,
    	cast(extract(month from scheduled_arrival) as int) as scheduled_arrival_month,
    	cast(extract(year from scheduled_arrival) as int) as scheduled_arrival_year,
    	cast(extract(dow from scheduled_arrival) as int) as scheduled_arrival_dow,
    	strftime('%B', scheduled_arrival) as scheduled_arrival_month_name,
    	strftime('%A', scheduled_arrival) as scheduled_arrival_day_name,
    	cast(scheduled_arrival as date) as scheduled_arrival_date,

    	-- actual arrival timestamp breakdown
    	cast(extract(day from actual_arrival) as int) as actual_arrival_day,
    	cast(extract(month from actual_arrival) as int) as actual_arrival_month,
    	cast(extract(year from actual_arrival) as int) as actual_arrival_year,
    	cast(extract(dow from actual_arrival) as int) as actual_arrival_dow,
    	strftime('%B', actual_arrival) as actual_arrival_month_name,
    	strftime('%A', actual_arrival) as actual_arrival_day_name,
    	cast(actual_arrival as date) as actual_arrival_date

	from stg_05_dim_datetime
    """
).df()


In [20]:
stg_06_dim_datetime_breakdown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284521 entries, 0 to 284520
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   sk_id                           284521 non-null  object        
 1   scheduled_departure_day         284521 non-null  int32         
 2   scheduled_departure_month       284521 non-null  int32         
 3   scheduled_departure_year        284521 non-null  int32         
 4   scheduled_departure_dow         284521 non-null  int32         
 5   scheduled_departure_month_name  284521 non-null  object        
 6   scheduled_departure_day_name    284521 non-null  object        
 7   scheduled_departure_date        284521 non-null  datetime64[us]
 8   actual_departure_day            234333 non-null  float64       
 9   actual_departure_month          234333 non-null  float64       
 10  actual_departure_year           234333 non-null  float64

In [21]:
stg_08_fact_table = db.sql(
    """
    select
        a.sk_id,
        a.flight_number,
        a.airline_icao,
        a.passenger_count,
        a.mod_flag,
        b1.airport_icao as orig_airport_icao,
        b2.airport_icao as dest_airport_icao,
        d.line_type_code,
        e.status_id as flight_status,
        f.departure_situation,
        f.arrival_situation,
        f.sk_id as flight_schedule_id

        
    from stg_03_convert_types_keys          as a
    left join dim_airports                  as b1   on a.orig_airport_icao = b1.airport_icao
    left join dim_airports                  as b2   on a.dest_airport_icao = b2.airport_icao
    left join dim_line_types                as d    on a.line_type_code = d.line_type_code
    left join dim_flight_status             as e    on a.flight_status = e.description
    left join stg_05_dim_datetime           as f   on a.sk_id = f.sk_id
    
    order by a.sk_id asc
    """
).df()

In [22]:
stg_08_fact_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284521 entries, 0 to 284520
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sk_id                284521 non-null  object
 1   flight_number        284521 non-null  object
 2   airline_icao         284521 non-null  object
 3   passenger_count      284521 non-null  int32 
 4   mod_flag             284521 non-null  int32 
 5   orig_airport_icao    284521 non-null  object
 6   dest_airport_icao    284521 non-null  object
 7   line_type_code       284521 non-null  object
 8   flight_status        284521 non-null  int64 
 9   departure_situation  284521 non-null  object
 10  arrival_situation    284521 non-null  object
 11  flight_schedule_id   284521 non-null  object
dtypes: int32(2), int64(1), object(9)
memory usage: 23.9+ MB


In [ ]:
azu_flights_records = db.sql(
    """
    select
        a.sk_id,
        a.flight_number,
        a.passenger_count,
        a.mod_flag,
        a.orig_airport_icao,
        b1.airport_name as orig_airport_name,
        b1.country as orig_airport_country,
        a.dest_airport_icao,
        b2.airport_name as dest_airport_name,
        b2.country as dest_airport_country,
        d.description as line_type_desc,
        e.description as flight_status_desc,
        --departure details
        g.scheduled_departure,
        g.actual_departure,
        g.departure_time_diff,
        f1.situation_short_desc as departure_situation,
        h.scheduled_departure_day,
        h.scheduled_departure_month,
        h.scheduled_departure_year,
        h.scheduled_departure_dow,
        h.scheduled_departure_month_name,
        h.scheduled_departure_day_name,
        h.scheduled_departure_date,
        h.actual_departure_day,
        h.actual_departure_month,
        h.actual_departure_year,
        h.actual_departure_dow,
        h.actual_departure_month_name,
        h.actual_departure_day_name,
        h.actual_departure_date,        
        --arrival details
        g.scheduled_arrival,
        g.actual_arrival,
        g.arrival_time_diff,
        f2.situation_short_desc as arrival_situation,
        h.scheduled_arrival_day,
        h.scheduled_arrival_month,
        h.scheduled_arrival_year,
        h.scheduled_arrival_dow,
        h.scheduled_arrival_month_name,
        h.scheduled_arrival_day_name,
        h.scheduled_arrival_date,
        h.actual_arrival_day,
        h.actual_arrival_month,
        h.actual_arrival_year,
        h.actual_arrival_dow,
        h.actual_arrival_month_name,
        h.actual_arrival_day_name,
        h.actual_arrival_date,        
        
        --flight time calculation
        g.expected_flight_duration,
        g.actual_flight_duration
        
    from stg_10_fact_table as a
    left join dim_airports as b1 on a.orig_airport_icao = b1.airport_icao
    left join dim_airports as b2 on a.dest_airport_icao = b2.airport_icao
    left join dim_line_types as d on a.line_type_code = d.line_type_code
    left join dim_flight_status as e on a.flight_status = e.status_id
    left join dim_flight_situation as f1 on a.departure_situation = f1.situation_id
    left join dim_flight_situation as f2 on a.arrival_situation = f2.situation_id
    left join stg_05_dim_datetime as g on a.flight_schedule_id = g.sk_id
    left join stg_06_dim_datetime_breakdown as h on a.flight_schedule_id = h.sk_id

    
    order by g.scheduled_departure asc
    """
    
).df()